In [75]:
import pandas as pd


class CFG:
    start_year = 1950
    end_year = 2100
    step_year = 10
    
    start_year_Timestamp = pd.Timestamp(f'{start_year}-01-01')
    end_year_Timestamp = pd.Timestamp(f'{end_year}-01-01')
    

# Retrieve nc file concat to pandas

    - walking scrapping/datas load nc file in xarray
    - xarray to pandas
    - concat each to global pandas dataframe ( restrainned on box(35:60,-3:12) )
    - make geometry column as Point(latitude,longitude)
    - pandas to geopandas
    - plot vars 

     41.3030:51.1242,
                                                                -5.4530:9.5610,

In [76]:
import xarray as xr
import pandas as pd
import os

def nc2pandas(path:str)->pd.DataFrame:
    df = pd.DataFrame()
    for file in list( os.walk('./datas/'))[0][2]:
        file_path = path + file.split('/')[0]
        print(file_path, end='...')
        try:
            df = pd.concat(
                [
                    df,
                    xr.open_dataset(file_path).to_dataframe().loc[
                                                                30:60,
                                                                -10:90,
                                                                CFG.start_year_Timestamp:CFG.end_year_Timestamp:CFG.step_year
                    ]
                ],axis=1)
            print('done')
        except:
            print('discarded.')
            
    return df      #.loc[:,:,CFG.start_year_Timestamp:CFG.end_year_Timestamp:CFG.step_year]

datas = nc2pandas('./datas/')
with pd.option_context('display.max_columns',None):
    display(datas.info())

./datas/BIO08_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...discarded.
./datas/BIO09_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...discarded.
./datas/BIO10_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...discarded.
./datas/BIO12_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/potential-evaporation_annual-mean_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO13_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO01_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO04_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO05_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO18_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...discarded.
./datas/BIO06_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO03_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...done
./datas/BIO11_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...discarded.
./datas/volumetric-soil-water-layer-1_annual-mean_ipsl-cm5a-lr_rcp85_r1i1p1_1950-2100_v1.0.nc...d

None

In [77]:
import geopandas as gpd
from shapely import Point

def geometrize(df:pd.DataFrame) -> gpd.GeoDataFrame:
    df.reset_index(inplace=True)
    display(df.head(1))
    df['geometry'] = [ Point(y,x) for x,y in zip (df.latitude.values, df.longitude.values)]
    # df.drop(['latitude','longitude'], axis=1, inplace=True)
    df.set_index('time', inplace=True)
    df = df.rename(columns=
                     {
                         'latitude': 'x',
                         'longitude': 'y',
                        'BIO01':'mean_temperature', 
                        'BIO05':'Maximum_temperature_of_warmest_month' , 
                        'BIO03':'Isothermality', 
                        'BIO06':'Minimum_temperature_of_coldest_month', 
                        'BIO07':'Temperature_annual_range', 
                        'BIO12':'Annual_precipitation', 
                        'BIO15':'Precipitation_seasonality',
                        'BIO13':'Precipitation_of_wettest_month',
                        'BIO02':'Mean_diurnal_range',
                        
                        'BIO14':'Precipitation_of_driest_month', 
                        'BIO04':'Temperature_seasonality'
                     }
                    )
    df = gpd.GeoDataFrame(df)
    years = int((CFG.end_year - CFG.start_year)/CFG.step_year)
    print(f'{years} years indicators for {int(df.shape[0]/years)} geolocalized points.')
    return df

datas = geometrize(datas)  
with pd.option_context('display.max_columns',None):
    display(datas.info())



,latitude,longitude,time,BIO12,potential-evaporation_annual-mean,BIO13,BIO01,BIO04,BIO05,BIO06,BIO03,volumetric-soil-water-layer-1_annual-mean,BIO02,BIO14,BIO15,BIO07
0,30.25,-9.75,1980-01-01,8.519572e-09,1.585844e-08,4.573391e-08,290.90271,174.292709,294.994781,289.493011,50.167217,0.0,2.913012,2.487088e-10,1.602787,5.501784


24 years indicators for 12500 geolocalized points.
<class 'geopandas.geodataframe.GeoDataFrame'>
DatetimeIndex: 300000 entries, 1980-01-01 to 2100-01-01
Data columns (total 16 columns):
 #   Column                                     Non-Null Count   Dtype   
---  ------                                     --------------   -----   
 0   x                                          300000 non-null  float64 
 1   y                                          300000 non-null  float64 
 2   Annual_precipitation                       300000 non-null  float32 
 3   potential-evaporation_annual-mean          300000 non-null  float32 
 4   Precipitation_of_wettest_month             300000 non-null  float32 
 5   mean_temperature                           300000 non-null  float32 
 6   Temperature_seasonality                    300000 non-null  float32 
 7   Maximum_temperature_of_warmest_month       300000 non-null  float32 
 8   Minimum_temperature_of_coldest_month       300000 non-null  float32 


None

In [78]:
import numpy as np
import joblib

def make_score(_datas):
    # Your wheat requirements
    wheat_requirements = {
        'mean_temperature': 291.761,
        'Maximum_temperature_of_warmest_month': 298.15,
        'Minimum_temperature_of_coldest_month': 268.15,
        'Precipitation_of_driest_month': 1.52e-07,
        'Precipitation_of_wettest_month':2.28e-07,
        'Annual_precipitation': 4.253e-07,
    }
    
    # Convert the wheat_requirements dictionary to a DataFrame
    wheat_df = pd.DataFrame(wheat_requirements, index=[0])

    df = pd.DataFrame(_datas.drop('geometry',axis=1))

    squared_diff = (df - wheat_df.iloc[0]) 
    
    for col in set(squared_diff.columns) - set(['x','y']):
        squared_diff[col] = squared_diff[col].rank(pct=True) **2

    # Sum of squared differences for each row
    sum_squared_diff = squared_diff.sum(axis=1)
    
    # Take the square root of the sum of squared differences
    _datas['norm'] = np.sqrt(sum_squared_diff)

    _datas.norm = _datas.norm.rank(pct=True)
    
    return _datas.loc[:,['x','y','norm','geometry']]


_folium_datas = make_score(datas)

# Dump on file for sharing

joblib.dump(_folium_datas, f'./scores/Bio_score_{CFG.start_year}to{CFG.end_year}step{CFG.step_year}.pickle')

_folium_datas.describe()

,x,y,norm
count,300000.000000,300000.000000,300000.000000
mean,45.000000,40.000000,0.500002
std,8.659066,28.867201,0.288676
min,30.250000,-9.750000,0.000003
25%,37.625000,15.125000,0.250003
50%,45.000000,40.000000,0.500002
75%,52.375000,64.875000,0.750001
max,59.750000,89.750000,1.000000
